In [1]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import urllib2
from multiprocessing import Pool as ThreadPool
import math
import cPickle
import os 

In [8]:
priceArr = []
patCollect = []
endingInd = []
patLen = 10
futureE = 10
quoteCollect = {}
bestArr = []

def percentChange(startPoint,currentPoint):
    try:
        x = ((float(currentPoint)-startPoint)/abs(startPoint))
        return x
    except:
        print(currentPoint, startPoint)
        return 0.00000000001

def loadPats():
    global priceArr

    priceArr = []
    with open(r"one.pickle", "rb") as input_file:
        e = cPickle.load(input_file)
        priceArr = e


def collectPats():
    global patCollect
    global endingInd
    patCollect = []
    endingInd = []
    for each in priceArr:
        tempCollect = []
        tempEnd = []
        sIndex = 0
        length = (len(each) - patLen - futureE)

        while sIndex < length:
            inc = 1
            tempPat = []
            while inc <= patLen:
                temp = percentChange(each[sIndex + inc -1], each[sIndex + inc])
                inc += 1
                tempPat.append(temp)
            tempCollect.append(tempPat)
            tempEnd.append(sIndex+patLen)
            sIndex += (patLen)
        patCollect.append(tempCollect)
        endingInd.append(tempEnd)

loadPats()
collectPats()

In [20]:
def yahooLoad(val):
    try:
        tempArr = []
        string = ""
        string = 'http://ichart.finance.yahoo.com/table.csv?s={0}'.format(val)
        count = 0
        for line in urllib2.urlopen(string):
            if count == 0:
                count = 1
                continue
            close = line.split(',')[6]
            close = float(str(close))
            tempArr.append(close)
        tempArr = tempArr[::-1]
        #quoteCollect[val] = tempArr
        return {val: tempArr[-100:]}
    except:
        {'error': 'error'}
        pass


def currentPat(curArr):
    global patLen
    curPat = []
    sliceLen = patLen
    curr = curArr[-(sliceLen+1):]
    while curr[-2] == curr[-1]:
        sliceLen += 1
        curr = curArr[-(sliceLen+1):]
    i = 0
    while i < patLen:
        temp = percentChange(curr[i], curr[i + 1])
        curPat.append(temp)
        i += 1
    return curPat


def sortPats(array):
    finalArr = []
    def getKey(tup):
        return tup[1]
    finalArr = sorted(array, key=getKey)
    return finalArr


# In[7]:

def matchPats(patCollect,endingInd,curPat):
    matchedPat = []
    matchedEndInd = []
    bestMatches = []
    for colInd,col in enumerate(patCollect):
        for rowInd,row in enumerate(col):
            mseCollect = []
            for ind,item in enumerate(row):
                mseTemp = abs(curPat[ind] - item)
                mseCollect.append(mseTemp)
            #mseAvg = np.average(weighting)
            mseAvg = np.average(mseCollect)
            bestMatches.append(([row,colInd,endingInd[colInd][rowInd]],mseAvg))
    test = sortPats(bestMatches)
    for item in test[:10]:
        matchedPat.append(item[0][0])
        matchedEndInd.append((item[0][1],item[0][2]))
    return matchedPat,matchedEndInd

In [21]:
def plotting(matchedPat,matchedEndInd):
    global futureE
    futureAverages = []
    stDev = []
    futureLine = []
    futurePercent = []
    if len(matchedPat) > 1:
        for i in matchedPat:
            tempPercent = []
            refInd = matchedPat.index(i)
            endInd = matchedEndInd[refInd]
            col = endInd[0]
            outcomesZ = priceArr[col][endInd[1]:(endInd[1]+patLen)]
            futureLine.append(outcomesZ)
            for x in outcomesZ:
                change = percentChange(priceArr[col][endInd[1]],x)
                tempPercent.append(change)
            futurePercent.append(tempPercent)

        futureAverages = []

        for arrItemNum in range(len(futurePercent[0])):
            tempOutcomes = []
            for arr in futurePercent:
                tempOutcomes.append(arr[arrItemNum])
            futureAverages.append(np.mean(tempOutcomes))
            stDev.append(np.std(tempOutcomes))

        return futureAverages,stDev


In [22]:
def sortBest(array):
    finalArr = []
    def getKey(val):
        return val['future'][-1]
    finalArr = sorted(array, key=getKey, reverse=True)
    return finalArr

bestCollect = []

def bestGo(val,curArr):
    global bestCollect
    try:
        #curArr = yahooLoad(val)
        curPat = currentPat(curArr)
        #matchedPat,matchedEndInd = matchPats(patCollect,endingInd,curPat)
        #futureAverages,stDev = plotting(matchedPat,matchedEndInd)
        #totalDict = {'symbol': val,'matches': matchedPat,'current': curPat,'future': futureAverages, 'stDev': stDev, 'sharpe': futureAverages[-1]/stDev[-1]}
        #bestCollect.append(totalDict)
    except:
        bestCollect.append({'error':'error'})
        pass

In [23]:
arr = ['MMM','ABT','ABBV','ACN','ATVI','AYI','ADBE','AAP','AES','AET','AMG','AFL','A','GAS','APD','AKAM','ALK','AA','ALXN','ALLE','AGN','ADS','ALL','GOOGL','GOOG','MO','AMZN','AEE','AAL','AEP','AXP','AIG','AMT','AWK','AMP','ABC','AME','AMGN','APH','APC','ADI','ANTM','AON','APA','AIV','AAPL','AMAT','ADM','AJG','AIZ','T','ADSK','ADP','AN','AZO','AVGO','AVB','AVY','BHI','BLL','BAC','BCR','BAX','BBT','BDX','BBBY','BRK-B','BBY','BIIB','BLK','HRB','BA','BWA','BXP','BSX','BMY','BF-B']

In [24]:
pool = ThreadPool(4)
quoteCollect = pool.map(yahooLoad, arr)
pool.close()
pool.join()

bestCollect = []
for i in quoteCollect:
    for key,value in i.iteritems():
        print(key,value)
        if value != 'error':
            bestGo(key,value)



('MMM', [150.424306, 151.331663, 151.36125, 151.785334, 152.613804, 151.441597, 149.673373, 152.941619, 154.501226, 155.584013, 155.742958, 155.415139, 156.656871, 155.136992, 156.18005, 157.938346, 157.223107, 155.832359, 158.534368, 158.355566, 158.325766, 159.011204, 159.507897, 159.140334, 159.031061, 158.832387, 160.809229, 160.739685, 161.335722, 161.812543, 162.666855, 164.236406, 163.749642, 163.819186, 163.18342, 163.372165, 165.180119, 163.302621, 165.647011, 165.52781, 166.421851, 165.29932, 164.713227, 165.706612, 166.054303, 165.52781, 165.488066, 166.918543, 167.474837, 167.047688, 167.663582, 168.428493, 168.527823, 167.822528, 167.544381, 167.544381, 167.266234, 165.080774, 166.650325, 165.895358, 166.272849, 167.22649, 166.858943, 166.054303, 166.660269, 167.772855, 167.405308, 169.133788, 168.647039, 169.024515, 167.206634, 168.428493, 166.699997, 166.820007, 165.110001, 165.009995, 166.110001, 168.630005, 169.990005, 168.910004, 168.889999, 168.320007, 168.690002, 16

In [10]:
bestCollect = []
for i in quoteCollect:
    for key,value in i.iteritems():
        bestGo(key,value)

finalBest = []
for val in bestCollect:
    if val['sharpe'] > 0.75:
        finalBest.append(val)

bestArr = sortBest(finalBest)

#os.remove("best.pickle")
with open(r"best.pickle", "wb") as output_file:
    cPickle.dump(bestArr, output_file)

In [11]:
t = []
with open(r"best.pickle", "rb") as input_file:
        t = cPickle.load(input_file)
print(t)

[{'matches': [[-0.016976869084129907, -0.024180633600434875, -0.005309268978070864, -0.005337439453052913, -0.0035789100679754823, -0.017952199928263298, 0.00182758765001768, -0.003649715676739673, 0.00549402095770354, 0.01639390022299873], [-0.016760736071312645, -0.0200034711667343, 0.00443733129102164, -0.004064366543725515, 0.00035488046613383587, -0.004434183303642848, -0.0021379692902386035, -0.001606806527708366, 0.018240347861595733, 0.01246934101824258], [-0.009956690119990735, -0.001311791028041323, -0.002626947475467547, 0.0035118489742860795, 0.00021870060558951292, -0.005466883882903786, 0.010554079516460224, 0.001958241050945184, 0.004994547919132336, 0.012748503575860269], [-0.006422543558848566, -0.024240464633469145, -0.002318648482323677, -0.0029880477095601643, -0.010989010623076513, 0.001346767631421902, 0.008742434431741814, -0.005333333333333338, 0.0130697050938338, 0.009923949718822302], [-0.013783873754018348, -0.014427406659234981, 0.004345814920249352, -0.0025

In [ ]:
print(quoteCollect[0])